In [1]:
import socket
import os

In [7]:
# Fixing the issue for downloading from a subdirectory with path (it doesn't change cwd to that subdirectory)

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

serverAddress = ('localhost', 2121)
sock.bind(serverAddress)
sock.listen(1)

server_directory = r"C:\Users\parin\Desktop\NetworkFinalProject\serverFolder"
client_cwd = server_directory

def search_file_recursive(target_file):
    root_directory = server_directory
    for current_dir, _, _ in os.walk(root_directory):
            target_file_path = os.path.join(current_dir, target_file)
            if os.path.isfile(target_file_path):
                print(f"Found '{target_file}' in: {current_dir}")
                client_cwd = current_dir
                print('Current working directory is', client_cwd)
                return target_file_path, current_dir
    return False

while True:
    print('Waiting for a connection...')
    connection, clientAddress = sock.accept()
    print('Connection from', clientAddress)

    # For receiving data
    while True:
        data = connection.recv(1024)
        print('\nReceived', data.decode())

        if data:
            if data.decode()[:6] == 'DELETE':
                path = data[8:len(data)-1].strip()
                # Find the path file and delete the file
                try:
                    os.remove(path)
                    print('File deleted:', path)
                except OSError as e:
                    print(f"Error deleting file {path}: {e}")
            elif data.decode()[:8] == 'DOWNLOAD':
                # Download using only the path
                sock2 = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                serverAddress2 = ('localhost', 2222)
                sock2.bind(serverAddress2)
                sock2.listen(5)
                print('Listening...')
                newConn, newAddr = sock2.accept()
                print('Got connection from', newAddr)
                file_path = data.decode()[9:]
                if os.path.exists(file_path) and file_path.find("serverFolder"):
                    client_cwd = server_directory
                    print('Sending')
                    with open(file_path, 'rb') as file:
                        while True:
                            file_data = file.read(1024)
                            if not file_data: break
                            newConn.sendall(file_data)
                elif search_file_recursive(file_name):
                    file_loc, client_cwd = search_file_recursive(file_name)
                    if os.path.exists(file_loc):
                        with open(file_loc, 'rb') as file:
                            while True:
                                file_data = file.read(1024)
                                if not file_data: break
                                newConn.sendall(file_data)
                else:
                    print(file_path + ' Not Found.')
                    newConn.sendall(b"File not found")
                newConn.close()
                sock2.close()

                # Download using only the name
#                 sock2 = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
#                 serverAddress2 = ('localhost', 2222)
#                 sock2.bind(serverAddress2)
#                 sock2.listen(5)
#                 print('Listening...')
#                 newConn, newAddr = sock2.accept()
#                 print('Got connection from', newAddr)
#                 file_name = data.decode()[9:]
#                 file_path = os.path.join(server_directory, file_name)
#                 if os.path.exists(file_path):
#                     print('Sending')
#                     with open(file_path, 'rb') as file:
#                         while True:
#                             file_data = file.read(1024)
#                             if not file_data: break
#                             newConn.sendall(file_data)
#                 elif search_file_recursive(file_name):
#                     file_loc, client_cwd = search_file_recursive(file_name)
#                     if os.path.exists(file_loc):
#                         with open(file_loc, 'rb') as file:
#                             while True:
#                                 file_data = file.read(1024)
#                                 if not file_data: break
#                                 newConn.sendall(file_data)
#                 else:
#                     print(file_path + ' Not Found.')
#                     newConn.sendall(b"File not found")
#                 newConn.close()
#                 sock2.close()
                    
            elif data.decode()[:4] == 'LIST':
                directory_path = server_directory
                file_list = []
                for root, dirs, files in os.walk(directory_path):
                    for filename in files:
                        file_path = os.path.join(root, filename)
                        file_size = os.path.getsize(file_path)
                        file_info = f"{filename} - {file_size} bytes"
                        file_list.append(file_info)
                if not file_list:
                    print(f"Directory not found: {directory_path}")
                    connection.sendall("Directory not found".encode())
                else:
                    print('Sending file list:', file_list)
                    connection.sendall('\n'.join(file_list).encode())
                    
            elif data.decode()[:3] == 'PWD':
                connection.sendall(client_cwd.encode())
                
            elif data.decode()[:6] == 'UPLOAD':
                client_cwd = server_directory
                file_name = data.decode()[7:]
                sock2 = socket.socket()
                sock2.bind(('localhost', 2222))
                sock2.listen(5)
                print('Listening...')
                newConn, newAddr = sock2.accept()
                print('Got connection from', newAddr)
                server_folder = r"C:\Users\parin\Desktop\NetworkFinalProject\serverFolder"
                file_path = os.path.join(server_folder, file_name)
                with open(file_path, 'wb') as file:
                    while True:
                        file_data = newConn.recv(1024)
                        if not file_data: break
                        file.write(file_data)
                    print('---DONE---')
                newConn.close()
                sock2.close()
                
            elif data == 'SHUTDOWN'.encode(): # When the client enters 'shutdown', the server shuts down
                break
        else:
            print('No more data from', clientAddress)
            break
    connection.close()
    if data == 'SHUTDOWN'.encode(): break
        
print('Shutting down the server.')        
sock.close()

Waiting for a connection...
Connection from ('127.0.0.1', 58205)

Received LIST
Sending file list: ['ALargeFile.txt - 5056506 bytes', 'anotherThingToUpload.txt - 9 bytes', 'clientFileToUpload.txt - 12 bytes', 'Final_Project.pdf - 267787 bytes', 'forClientToDownload.txt - 26 bytes', 'FUM_OS_1402-01_Project2.pdf - 0 bytes', 'output2_fcfs.txt - 1000 bytes', 'ParinazAkef-NetworkMidtermProject-4002262224.pdf - 1442024 bytes', 'somethingElseInthisClientDirectory.txt - 11 bytes', 'zForTesting1.txt - 0 bytes', 'zForTesting2.txt - 0 bytes', 'zForTesting3 - Copy.txt - 3 bytes', 'zForTesting3.txt - 0 bytes', 'zForTesting5.txt - 26 bytes', 'AfileTotestDifferentDirectory.txt - 98010 bytes', 'fromTheServerDirectorytoDownload.txt - 15 bytes', 'inAdirectoryFolder.txt - 0 bytes', 'output2_rr.txt - 1040 bytes', 'output2_sjf.txt - 1057 bytes']

Received PWD

Received SHUTDOWN
Shutting down the server.
